In [1]:
import os 
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm

sectors = ['agriculture', 'apparel', 'chemicals', 'machinery', 'materials', 
           'metals', 'miscellaneous', 'transport', 'minerals']

new_sectors = ['Agriculture', 'Apparel', 'Chemicals', 'Machinery', 'Materials', 
           'Metals', 'Miscellaneous', 'Transport', 'Minerals']
rename_columns = {s: ns for s, ns in zip(sectors, new_sectors)}
sectors = new_sectors

region2agg_dict = {
    'east africa': 'Africa', 
    'central africa': 'Africa',
    'north africa': 'Africa',
    'southern africa': 'Africa',
    'west africa': 'Africa',
    'caribbean': 'America',
    'central america': 'America',
    'south america': 'America',
    'north america': 'America',
    'central asia': 'Asia',
    'east asia': 'Asia',
    'south asia': 'Asia',
    'southeast asia': 'Asia',
    'west asia': 'Asia',
    'eastern europe': 'Europe', 
    'nordic': 'Europe',
    'southern europe': 'Europe',
    'western europe': 'Europe',
    'australia and new zealand': 'Oceania',
    'melanesia': 'Oceania',
    'micronesia': 'Oceania',
    'polynesia': 'Oceania',   
}


project_dir = Path('C:\shares_TARIFF_project\comtrade_new\DID_country')


volume_df = pd.read_csv(project_dir / '2015_2021.csv', header=0)
volume_df = volume_df.rename(columns=rename_columns)
volume_df.head()

,source,year,month,Agriculture,Apparel,Chemicals,Machinery,Materials,Metals,Minerals,Miscellaneous,Transport
0,Afghanistan,2015,1,807693.0,508764.0,NaN,45445.0,NaN,4956.0,NaN,269303.0,20858.0
1,Afghanistan,2015,2,564296.0,170889.0,NaN,47867.0,NaN,4838991.0,NaN,403538.0,NaN
2,Afghanistan,2015,3,153039.0,731135.0,NaN,5582.0,NaN,146989.0,NaN,279085.0,NaN
3,Afghanistan,2015,4,384263.0,281741.0,NaN,26328.0,2856.0,474628.0,NaN,1070675.0,4451.0
4,Afghanistan,2015,5,57945.0,240599.0,NaN,10655.0,24119.0,1664127.0,NaN,676760.0,3607.0


In [2]:
region_df = pd.read_csv(project_dir / 'countries2regions_new.csv', header=0)
country2region_dict = {c:r for c, r in zip(region_df['source_country'], region_df['source_region'])}

region_df['agg'] = region_df['source_region'].map(region2agg_dict)

region2countries_dict = {}
for region, group_df in region_df.groupby(by=['agg']):
    region2countries_dict[region] = group_df['source_country'].tolist()

# 区分 USA和 Rest of America(ROA)

China_list = ['China']
region2countries_dict['China'] = China_list
roa_list = []
for c in region2countries_dict.get('Asia'):
    if c not in China_list:
        roa_list.append(c)
region2countries_dict['ROA'] = roa_list
del region2countries_dict['Asia']


C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\2890156325.py:7: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for region, group_df in region_df.groupby(by=['agg']):


# get panel data - trade war

In [3]:
country2region_dict = {}
for rg, ct_list in region2countries_dict.items():
    for ct in ct_list:
        country2region_dict[ct] = rg

# for sector in sectors:
#     volume_df[sector] = np.log(volume_df[sector] + 1)

volume_df['region'] = volume_df['source'].map(country2region_dict)
volume_sum_df = volume_df.groupby(by=['region', 'year', 'month']).sum()

volume_sum_df['region'], volume_sum_df['year'], volume_sum_df['month'] = zip(*volume_sum_df.index)
volume_sum_df.reset_index(drop=True, inplace=True)
#print(volume_sum_df.head())
#volume_sum_df.to_csv(r'C:\\Users\\Ki Siyuan\\Desktop\\vol_sum.csv')
# volume_mean_df = volume_sum_df.groupby(by=['year', 'month']).mean()
# volume_mean_df['year'], volume_mean_df['month'] = zip(*volume_mean_df.index)
# volume_mean_df.reset_index(drop=True, inplace=True)


# volume => percentage 转化为占比
years = volume_sum_df['year'].unique()
months = volume_sum_df['month'].unique()
for y in years:
    for m in months:
        query_df = volume_sum_df.query('year == @y and month == @m')[sectors]
        _sum = query_df.sum()
        secter2sum_dict = {s:v for s, v in zip(_sum.index, _sum.tolist())}
        for idx in query_df.index:
            for s, v in secter2sum_dict.items():
                _v = volume_sum_df.loc[idx, s]
                p = _v / v * 100
                volume_sum_df.loc[idx, s] = p


C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\136703564.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_sum_df = volume_df.groupby(by=['region', 'year', 'month']).sum()


In [4]:
# empty dir
panel_dir = project_dir / 'panels-trade-war'
for fp in panel_dir.rglob('*.csv'):
    os.remove(fp)

In [5]:
exp_years = [2018,2019]
com_years = [2015,2016,2017]
# sector = 'agriculture'
# region = 'China'

# sector = 'agriculture'
# region = 'America'


def get_did_panel(volume_sum_df, exp_years, com_years, sector, region):
    # volume_sum_df 中除去region外的求平均
    volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
    volume_mean_df['year'], volume_mean_df['month'] = zip(*volume_mean_df.index)
    volume_mean_df.reset_index(drop=True, inplace=True)
    
    com_df = volume_mean_df.query('year < 2020')[['year', 'month', sector]]
    com_df['name'] = 'mean'
    com_df.loc[com_df.query('year in @com_years').index, 'time'] = 0
    com_df.loc[com_df.query('year in @exp_years').index, 'time'] = 1
    com_df['time'] = com_df['time'].astype('int')
    com_df.loc[:, 'group'] = 0


    exp_df = volume_sum_df.query('year < 2020 and region == @region')[['region', 'year', 'month', sector]]
    exp_df = exp_df.rename(columns={'region': 'name'})
    exp_df.loc[exp_df.query('year in @com_years').index, 'time'] = 0
    exp_df.loc[exp_df.query('year in @exp_years').index, 'time'] = 1
    exp_df['time'] = exp_df['time'].astype('int')
    exp_df.loc[:, 'group'] = 1


    concat_df = pd.concat([exp_df, com_df])
    concat_df['did'] = concat_df.apply(lambda x: x['time'] * x['group'], axis=1)
    concat_df['date'] = concat_df.apply(lambda x: '{}-{}'.format(x['year'], x['month']), axis=1)
    del concat_df['year'], concat_df['month']
    concat_df = concat_df.rename(columns={sector: 'value'})
    # concat_df['value'] = np.log(concat_df['value'] + 1)
    return concat_df

for region in region2countries_dict.keys():
    for sector in sectors:
        panel_df = get_did_panel(volume_sum_df, exp_years, com_years, sector, region)
        panel_df.to_csv(panel_dir / f'{region}-{sector}.csv', header=True, index=False)

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\2219955876.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\2219955876.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\2219955876.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeri

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\2219955876.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\2219955876.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\2219955876.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeri

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\2219955876.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\2219955876.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()


# did-trade-war

In [6]:
import re
import ipystata

region_list = list(region2countries_dict.keys())
sector_list = sectors

# empty dir
panel_dir = project_dir / 'temp'
for fp in panel_dir.rglob('*.txt'):
    os.remove(fp)

IPyStata is loaded in batch mode.


In [7]:
%%stata -i sector_list -i region_list
foreach region in `region_list' {
    display "------------------------`region'---------------------------------"
    clear
    cd "C:\shares_TARIFF_project\comtrade_new\DID"
    foreach sector in `sector_list' {
        import delimited "./panels-trade-war/`region'-`sector'.csv"
        egen time1 = group(date)
        xtset time1 group
        reghdfe value did, absorb(time1 group)
        outreg2 using "./temp/tradewar-`region'-`sector'.txt"
        clear
    }
}


------------------------Africa---------------------------------
C:\shares_TARIFF_project\comtrade_new\DID
(6 vars, 120 obs)
       panel variable:  time1 (strongly balanced)
        time variable:  group, 0 to 1
                delta:  1 unit
(MWFE estimator converged in 2 iterations)

HDFE Linear regression                            Number of obs   =        120
Absorbing 2 HDFE groups                           F(   1,     58) =       0.22
                                                  Prob > F        =     0.6440
                                                  R-squared       =     0.9991
                                                  Adj R-squared   =     0.9981
                                                  Within R-sq.    =     0.0037
                                                  Root MSE        =     0.3833

------------------------------------------------------------------------------
       value |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]


In [8]:
def get_result(result_fp):
    with open(result_fp, 'r') as fp:
        result_str = fp.read()
        match_str = re.findall(r'did\t(.*?)\n', result_str)[0]
        star_count = match_str.count('*')
        match_str = match_str.replace(',', '')
        if match_str == '-':# 若匹配到空，判定不显著
            significant = -1
            coff = -999
        else:
            if star_count == 0:
                significant = -1
                coff = float(match_str)
            else:
                coff = float(match_str.split('*')[0])
                if star_count == 1:
                    significant = 0.1
                elif star_count == 2:
                    significant = 0.05
                else:
                    significant = 0.01
    return coff, significant

result_list = []
for region in region_list:
    for sector in sector_list:
        result_fp = project_dir / 'temp' / f'tradewar-{region}-{sector}.txt'
        coff, significant = get_result(result_fp)
        result_list.append((region, sector, coff, significant))
result_df = pd.DataFrame(result_list, columns=['region', 'sector', 'coff', 'significance'])



# # 将coff的值标准化
# data = result_df['coff']
# data_t = (data-data.mean())/(data.max()-data.min())
# result_df['coff'] = data_t
result_df.to_csv('C:\shares_TARIFF_project\comtrade_new\DID\did-result-trade-war-macro.csv', header=True, index=False)
# result_df

# get panel data-pandemic

In [9]:
country2region_dict = {}
for rg, ct_list in region2countries_dict.items():
    for ct in ct_list:
        country2region_dict[ct] = rg

# for sector in sectors:
#     volume_df[sector] = np.log(volume_df[sector] + 1)

volume_df['region'] = volume_df['source'].map(country2region_dict)
volume_sum_df = volume_df.groupby(by=['region', 'year', 'month']).sum()

volume_sum_df['region'], volume_sum_df['year'], volume_sum_df['month'] = zip(*volume_sum_df.index)
volume_sum_df.reset_index(drop=True, inplace=True)
#print(volume_sum_df.head())
#volume_sum_df.to_csv(r'C:\\Users\\Ki Siyuan\\Desktop\\vol_sum.csv')
# volume_mean_df = volume_sum_df.groupby(by=['year', 'month']).mean()
# volume_mean_df['year'], volume_mean_df['month'] = zip(*volume_mean_df.index)
# volume_mean_df.reset_index(drop=True, inplace=True)


# volume => percentage 转化为占比
years = volume_sum_df['year'].unique()
months = volume_sum_df['month'].unique()
for y in years:
    for m in months:
        query_df = volume_sum_df.query('year == @y and month == @m')[sectors]
        _sum = query_df.sum()
        secter2sum_dict = {s:v for s, v in zip(_sum.index, _sum.tolist())}
        for idx in query_df.index:
            for s, v in secter2sum_dict.items():
                _v = volume_sum_df.loc[idx, s]
                p = _v / v * 100
                volume_sum_df.loc[idx, s] = p


C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\136703564.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_sum_df = volume_df.groupby(by=['region', 'year', 'month']).sum()


In [10]:
# empty dir
panel_dir = project_dir / 'panels-pandemic'
for fp in panel_dir.rglob('*.csv'):
    os.remove(fp)

In [11]:
exp_years = [2020,2021]
com_years = [2018,2019]


def get_did_panel(volume_sum_df, exp_years, com_years, sector, region):
    # volume_sum_df 中除去region外的求平均
    volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
    volume_mean_df['year'], volume_mean_df['month'] = zip(*volume_mean_df.index)
    volume_mean_df.reset_index(drop=True, inplace=True)
    
    com_df = volume_mean_df[['year', 'month', sector]]
    com_df['name'] = 'mean'
    com_df.loc[com_df.query('year in @com_years').index, 'time'] = 0
    com_df.loc[com_df.query('year in @exp_years').index, 'time'] = 1
    com_df = com_df.dropna()
    com_df['time'] = com_df['time'].astype('int')
    com_df.loc[:, 'group'] = 0


    exp_df = volume_sum_df.query('region == @region')[['region', 'year', 'month', sector]]
    exp_df = exp_df.rename(columns={'region': 'name'})
    exp_df.loc[exp_df.query('year in @com_years').index, 'time'] = 0
    exp_df.loc[exp_df.query('year in @exp_years').index, 'time'] = 1
    exp_df = exp_df.dropna()
    exp_df['time'] = exp_df['time'].astype('int')
    exp_df.loc[:, 'group'] = 1


    concat_df = pd.concat([exp_df, com_df])
    concat_df['did'] = concat_df.apply(lambda x: x['time'] * x['group'], axis=1)
    concat_df['date'] = concat_df.apply(lambda x: '{}-{}'.format(x['year'], x['month']), axis=1)
    del concat_df['year'], concat_df['month']
    concat_df = concat_df.rename(columns={sector: 'value'})
    # concat_df['value'] = np.log(concat_df['value'] + 1)
    return concat_df


for region in region2countries_dict.keys():
    for sector in sectors:
        panel_df = get_did_panel(volume_sum_df, exp_years, com_years, sector, region)
        panel_df.to_csv(panel_dir / f'{region}-{sector}.csv', header=True, index=False)

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\488535082.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\488535082.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['name'] = 'mean'
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\488535082.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\488535082.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\488535082.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['name'] = 'mean'
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\488535082.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\488535082.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df['time'] = exp_df['time'].astype('int')
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\488535082.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df.loc[:, 'group'] = 1
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\488535082.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\488535082.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\488535082.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['name'] = 'mean'
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\488535082.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\488535082.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\488535082.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['name'] = 'mean'
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\488535082.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

# did-pandemic

In [12]:
import re
import ipystata

region_list = list(region2countries_dict.keys())
sector_list = sectors

# empty dir
panel_dir = project_dir / 'temp-1'
for fp in panel_dir.rglob('*.txt'):
    os.remove(fp)

In [13]:
%%stata -i sector_list -i region_list
foreach region in `region_list' {
    display "------------------------`region'---------------------------------"
    clear
    cd "C:\shares_TARIFF_project\comtrade_new\DID"
    foreach sector in `sector_list' {
        import delimited "./panels-pandemic/`region'-`sector'.csv"
        egen time1 = group(date)
        xtset time1 group
        reghdfe value did, absorb(time1 group)
        outreg2 using "./temp-1/pandemic-`region'-`sector'.txt"
        clear
    }
}


------------------------Africa---------------------------------
C:\shares_TARIFF_project\comtrade_new\DID
(6 vars, 96 obs)
       panel variable:  time1 (strongly balanced)
        time variable:  group, 0 to 1
                delta:  1 unit
(MWFE estimator converged in 2 iterations)

HDFE Linear regression                            Number of obs   =         96
Absorbing 2 HDFE groups                           F(   1,     46) =       1.30
                                                  Prob > F        =     0.2601
                                                  R-squared       =     0.9993
                                                  Adj R-squared   =     0.9986
                                                  Within R-sq.    =     0.0275
                                                  Root MSE        =     0.3304

------------------------------------------------------------------------------
       value |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-

In [14]:
def get_result(result_fp):
    with open(result_fp, 'r') as fp:
        result_str = fp.read()
        match_str = re.findall(r'did\t(.*?)\n', result_str)[0]
        star_count = match_str.count('*')
        match_str = match_str.replace(',', '')
        if match_str == '-':# 若匹配到空，判定不显著
            significant = -1
            coff = -999
        else:
            if star_count == 0:
                significant = -1
                coff = float(match_str)
            else:
                coff = float(match_str.split('*')[0])
                if star_count == 1:
                    significant = 0.1
                elif star_count == 2:
                    significant = 0.05
                else:
                    significant = 0.01
    return coff, significant

result_list = []
for region in region_list:
    for sector in sector_list:
        result_fp = project_dir / 'temp-1' / f'pandemic-{region}-{sector}.txt'
        coff, significant = get_result(result_fp)
        result_list.append((region, sector, coff, significant))
result_df = pd.DataFrame(result_list, columns=['region', 'sector', 'coff', 'significance'])



# # 将coff的值标准化
# data = result_df['coff']
# data_t = (data-data.mean())/(data.max()-data.min())
# result_df['coff'] = data_t
result_df.to_csv('C:\shares_TARIFF_project\comtrade_new\DID\did-result-pandemic-macro.csv', header=True, index=False)
# result_df

# get panel data -two events

In [15]:
country2region_dict = {}
for rg, ct_list in region2countries_dict.items():
    for ct in ct_list:
        country2region_dict[ct] = rg

# for sector in sectors:
#     volume_df[sector] = np.log(volume_df[sector] + 1)

volume_df['region'] = volume_df['source'].map(country2region_dict)
volume_sum_df = volume_df.groupby(by=['region', 'year', 'month']).sum()

volume_sum_df['region'], volume_sum_df['year'], volume_sum_df['month'] = zip(*volume_sum_df.index)
volume_sum_df.reset_index(drop=True, inplace=True)
#print(volume_sum_df.head())
#volume_sum_df.to_csv(r'C:\\Users\\Ki Siyuan\\Desktop\\vol_sum.csv')
# volume_mean_df = volume_sum_df.groupby(by=['year', 'month']).mean()
# volume_mean_df['year'], volume_mean_df['month'] = zip(*volume_mean_df.index)
# volume_mean_df.reset_index(drop=True, inplace=True)


# volume => percentage 转化为占比
years = volume_sum_df['year'].unique()
months = volume_sum_df['month'].unique()
for y in years:
    for m in months:
        query_df = volume_sum_df.query('year == @y and month == @m')[sectors]
        _sum = query_df.sum()
        secter2sum_dict = {s:v for s, v in zip(_sum.index, _sum.tolist())}
        for idx in query_df.index:
            for s, v in secter2sum_dict.items():
                _v = volume_sum_df.loc[idx, s]
                p = _v / v * 100
                volume_sum_df.loc[idx, s] = p


C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\136703564.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_sum_df = volume_df.groupby(by=['region', 'year', 'month']).sum()


In [16]:
# empty dir
panel_dir = project_dir / 'panels-two-events'
for fp in panel_dir.rglob('*.csv'):
    os.remove(fp)

In [17]:
exp_years = [2020,2021]
com_years = [2015,2016,2017]


def get_did_panel(volume_sum_df, exp_years, com_years, sector, region):
    # volume_sum_df 中除去region外的求平均
    volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
    volume_mean_df['year'], volume_mean_df['month'] = zip(*volume_mean_df.index)
    volume_mean_df.reset_index(drop=True, inplace=True)
    
    com_df = volume_mean_df[['year', 'month', sector]]
    com_df['name'] = 'mean'
    com_df.loc[com_df.query('year in @com_years').index, 'time'] = 0
    com_df.loc[com_df.query('year in @exp_years').index, 'time'] = 1
    com_df = com_df.dropna()
    com_df['time'] = com_df['time'].astype('int')
    com_df.loc[:, 'group'] = 0


    exp_df = volume_sum_df.query('region == @region')[['region', 'year', 'month', sector]]
    exp_df = exp_df.rename(columns={'region': 'name'})
    exp_df.loc[exp_df.query('year in @com_years').index, 'time'] = 0
    exp_df.loc[exp_df.query('year in @exp_years').index, 'time'] = 1
    exp_df = exp_df.dropna()
    exp_df['time'] = exp_df['time'].astype('int')
    exp_df.loc[:, 'group'] = 1


    concat_df = pd.concat([exp_df, com_df])
    concat_df['did'] = concat_df.apply(lambda x: x['time'] * x['group'], axis=1)
    concat_df['date'] = concat_df.apply(lambda x: '{}-{}'.format(x['year'], x['month']), axis=1)
    del concat_df['year'], concat_df['month']
    concat_df = concat_df.rename(columns={sector: 'value'})
    # concat_df['value'] = np.log(concat_df['value'] + 1)
    return concat_df


for region in region2countries_dict.keys():
    for sector in sectors:
        panel_df = get_did_panel(volume_sum_df, exp_years, com_years, sector, region)
        panel_df.to_csv(panel_dir / f'{region}-{sector}.csv', header=True, index=False)

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\4294350675.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\4294350675.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['name'] = 'mean'
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\4294350675.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\4294350675.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\4294350675.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['name'] = 'mean'
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\4294350675.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\4294350675.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\4294350675.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['name'] = 'mean'
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\4294350675.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\4294350675.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\4294350675.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['name'] = 'mean'
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\4294350675.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\4294350675.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\4294350675.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['name'] = 'mean'
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_15176\4294350675.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

# did-three-events

In [18]:
import re
import ipystata

region_list = list(region2countries_dict.keys())
sector_list = sectors

# empty dir
panel_dir = project_dir / 'temp-2'
for fp in panel_dir.rglob('*.txt'):
    os.remove(fp)

In [19]:
%%stata -i sector_list -i region_list
foreach region in `region_list' {
    display "------------------------`region'---------------------------------"
    clear
    cd "C:\shares_TARIFF_project\comtrade_new\DID"
    foreach sector in `sector_list' {
        import delimited "./panels-two-events/`region'-`sector'.csv"
        egen time1 = group(date)
        xtset time1 group
        reghdfe value did, absorb(time1 group)
        outreg2 using "./temp-2/two-events-`region'-`sector'.txt"
        clear
    }
}


------------------------Africa---------------------------------
C:\shares_TARIFF_project\comtrade_new\DID
(6 vars, 120 obs)
       panel variable:  time1 (strongly balanced)
        time variable:  group, 0 to 1
                delta:  1 unit
(MWFE estimator converged in 2 iterations)

HDFE Linear regression                            Number of obs   =        120
Absorbing 2 HDFE groups                           F(   1,     58) =       0.37
                                                  Prob > F        =     0.5436
                                                  R-squared       =     0.9991
                                                  Adj R-squared   =     0.9981
                                                  Within R-sq.    =     0.0064
                                                  Root MSE        =     0.3840

------------------------------------------------------------------------------
       value |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]


In [20]:
def get_result(result_fp):
    with open(result_fp, 'r') as fp:
        result_str = fp.read()
        match_str = re.findall(r'did\t(.*?)\n', result_str)[0]
        star_count = match_str.count('*')
        match_str = match_str.replace(',', '')
        if match_str == '-':# 若匹配到空，判定不显著
            significant = -1
            coff = -999
        else:
            if star_count == 0:
                significant = -1
                coff = float(match_str)
            else:
                coff = float(match_str.split('*')[0])
                if star_count == 1:
                    significant = 0.1
                elif star_count == 2:
                    significant = 0.05
                else:
                    significant = 0.01
    return coff, significant

result_list = []
for region in region_list:
    for sector in sector_list:
        result_fp = project_dir / 'temp-2' / f'two-events-{region}-{sector}.txt'
        coff, significant = get_result(result_fp)
        result_list.append((region, sector, coff, significant))
result_df = pd.DataFrame(result_list, columns=['region', 'sector', 'coff', 'significance'])



# # 将coff的值标准化
# data = result_df['coff']
# data_t = (data-data.mean())/(data.max()-data.min())
# result_df['coff'] = data_t
result_df.to_csv('C:\shares_TARIFF_project\comtrade_new\DID\did-result-two-events-macro.csv', header=True, index=False)
# result_df

In [22]:
result_two_events_fp = 'C:\shares_TARIFF_project\comtrade_new\DID\did-result-two-events-macro.csv'
result_two_events_df = pd.read_csv(result_two_events_fp)
result_pandemic_fp = 'C:\shares_TARIFF_project\comtrade_new\DID\did-result-pandemic-macro.csv'
result_pandemic_df = pd.read_csv(result_pandemic_fp)
result_trade_war_fp = 'C:\shares_TARIFF_project\comtrade_new\DID\did-result-trade-war-macro.csv'
result_trade_war_df = pd.read_csv(result_trade_war_fp)

import numpy as np

# 如果sign=-1（不显著），设置coff为NaN
def func(x):
    if x['significance'] == -1:
        return np.nan
    else:
        return x['coff']
result_two_events_df['coff'] = result_two_events_df.apply(func, axis=1)
result_pandemic_df['coff'] = result_pandemic_df.apply(func, axis=1)
result_trade_war_df['coff'] = result_trade_war_df.apply(func, axis=1)

merge_df = pd.merge(result_two_events_df,result_pandemic_df,  
         left_on=['region', 'sector'], 
         right_on=['region', 'sector'], 
         how='inner') \
    .rename(columns={'coff_x': 'coff_two_events', 
                     'coff_y': 'coff_pandemic',
                     'significance_x': 'sign_two_events', 
                     'significance_y': 'sign_pandemic'})


merge_df = pd.merge(merge_df, result_trade_war_df, 
         left_on=['region', 'sector'], 
         right_on=['region', 'sector'], 
         how='inner') \
    .rename(columns={'coff': 'coff_trade_war', 
                     'significance': 'sign_trade_war'})





# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# data = merge_df['coff_tradewar'].tolist()
# data_t = scaler.fit_transform(np.array(data).reshape(-1,1)).reshape(1,-1).tolist()[0]
# merge_df['coff_tradewar'] = data_t


# data = merge_df['coff_pandemic'].tolist()
# data_t = scaler.fit_transform(np.array(data).reshape(-1,1)).reshape(1,-1).tolist()[0]
# merge_df['coff_pandemic'] = data_t

merge_df.to_csv('C:\shares_TARIFF_project\comtrade_new\DID\did-result-merge-macro.csv', header=True, index=False)
merge_df.head()

,region,sector,coff_two_events,sign_two_events,coff_pandemic,sign_pandemic,coff_trade_war,sign_trade_war
0,Africa,Agriculture,NaN,-1.00,NaN,-1.00,NaN,-1.00
1,Africa,Apparel,0.5050,0.01,NaN,-1.00,0.464,0.01
2,Africa,Chemicals,NaN,-1.00,-0.2880,0.01,0.188,0.05
3,Africa,Machinery,-0.0174,0.05,-0.0173,0.05,NaN,-1.00
4,Africa,Materials,0.1970,0.01,0.0494,0.05,0.148,0.01


In [23]:
for coff_str in ['coff_two_events', 'coff_pandemic','coff_trade_war']:
    pivot_df = merge_df[['region', 'sector', coff_str]].set_index('region') \
        .pivot(columns='sector', values=coff_str)
    pivot_df.to_csv(f'C:\shares_TARIFF_project\comtrade_new\DID\pivot-{coff_str}-macro.csv', header=True, index=True)